In [1]:
import pandas as pd
import numpy as np

from source.make_train_test import make_teams_target

pd.set_option("max_columns", 300)

In [2]:
def _add_stage(total):
    total['stage'] = '68'
    total.loc[(total.DayNum == 136) | (total.DayNum == 136), 'stage'] = '64'
    total.loc[(total.DayNum == 138) | (total.DayNum == 139), 'stage'] = '32'
    total.loc[(total.DayNum == 143) | (total.DayNum == 144), 'stage'] = '16'
    total.loc[(total.DayNum == 145) | (total.DayNum == 146), 'stage'] = '8'
    total.loc[(total.DayNum == 152), 'stage'] = '4'
    total.loc[(total.DayNum == 154), 'stage'] = 'Final'

    #total = pd.get_dummies(total, columns=['stage'])

    #del total['stage_68']

    return total

In [3]:
playoff_compact = 'data/raw_men/MDataFiles_Stage1/MNCAATourneyCompactResults.csv'
seed = 'data/raw_men/MDataFiles_Stage1/MNCAATourneySeeds.csv'

In [4]:
seeds = pd.read_csv(seed)

seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [5]:
target_data = pd.read_csv(playoff_compact)
target_data = make_teams_target(target_data, 'men')
target_data.head()

,Season,DayNum,Team1,Team2,target,target_points,T1_Loc,T2_Loc,ID
0,2003,134,1411,1421,0,-8,0,0,2003_1411_1421
1,2003,136,1112,1436,1,29,0,0,2003_1112_1436
2,2003,136,1113,1272,1,13,0,0,2003_1113_1272
3,2003,136,1141,1166,1,6,0,0,2003_1141_1166
4,2003,136,1143,1301,1,2,0,0,2003_1143_1301


In [6]:
seeds['region'] = seeds['Seed'].apply(lambda x: x[0])
seeds['Seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds.head()

,Season,Seed,TeamID,region
0,1985,1,1207,W
1,1985,2,1210,W
2,1985,3,1228,W
3,1985,4,1260,W
4,1985,5,1374,W


In [7]:
target_data = pd.merge(target_data, seeds.rename(columns={'TeamID': 'Team1', 
                                                          'Seed': 'T1_Seed', 
                                                          'region': 'T1_region'}), 
                       on=['Season', 'Team1'], how='left')
target_data = pd.merge(target_data, seeds.rename(columns={'TeamID': 'Team2', 
                                                          'Seed': 'T2_Seed', 
                                                          'region': 'T2_region'}), 
                       on=['Season', 'Team2'], how='left')

target_data = target_data[target_data.DayNum >= 136].copy()

target_data.head(10)

,Season,DayNum,Team1,Team2,target,target_points,T1_Loc,T2_Loc,ID,T1_Seed,T1_region,T2_Seed,T2_region
1,2003,136,1112,1436,1,29,0,0,2003_1112_1436,1,Z,16,Z
2,2003,136,1113,1272,1,13,0,0,2003_1113_1272,10,Z,7,Z
3,2003,136,1141,1166,1,6,0,0,2003_1141_1166,11,Z,6,Z
4,2003,136,1143,1301,1,2,0,0,2003_1143_1301,8,W,9,W
5,2003,136,1140,1163,0,-5,0,0,2003_1140_1163,12,X,5,X
6,2003,136,1161,1181,0,-10,0,0,2003_1161_1181,14,Z,3,Z
7,2003,136,1153,1211,0,-5,0,0,2003_1153_1211,8,Z,9,Z
8,2003,136,1228,1443,1,5,0,0,2003_1228_1443,4,Z,13,Z
9,2003,136,1242,1429,1,3,0,0,2003_1242_1429,2,Z,15,Z
10,2003,136,1221,1266,0,-4,0,0,2003_1221_1266,14,Y,3,Y


In [8]:
target_data = _add_stage(target_data)

target_data.head(10)

target_data.rename(columns={'stage': 'real_stage'}, inplace=True)

In [9]:
target_data['Seed_diff'] = target_data['T1_Seed'] - target_data['T2_Seed']

hist = target_data.groupby(['T1_Seed', 'real_stage'], as_index=False).target.mean()

In [10]:
hist[hist.T1_Seed<8]

,T1_Seed,real_stage,target
0,1,16,0.810345
1,1,32,0.865672
2,1,4,0.680000
3,1,64,1.000000
4,1,68,0.972222
5,1,8,0.531915
6,1,Final,0.647059
7,2,16,0.714286
8,2,32,0.656250
9,2,4,0.428571


In [11]:
hist[hist.T1_Seed>=8]

,T1_Seed,real_stage,target
47,8,16,0.600000
48,8,32,0.142857
49,8,4,1.000000
50,8,64,0.531250
51,8,68,0.500000
52,8,8,0.666667
53,8,Final,0.000000
54,9,16,0.600000
55,9,32,0.151515
56,9,4,0.000000


In [12]:
def add_stage(data):
    data.loc[(data.T1_region == 'W') & (data.T2_region == 'X'), 'stage'] = 'finalfour'
    data.loc[(data.T1_region == 'X') & (data.T2_region == 'W'), 'stage'] = 'finalfour'
    data.loc[(data.T1_region == 'Y') & (data.T2_region == 'Z'), 'stage'] = 'finalfour'
    data.loc[(data.T1_region == 'Z') & (data.T2_region == 'Y'), 'stage'] = 'finalfour'
    data.loc[(data.T1_region == 'W') & (data.T2_region.isin(['Y', 'Z'])), 'stage'] = 'final'
    data.loc[(data.T1_region == 'X') & (data.T2_region.isin(['Y', 'Z'])), 'stage'] = 'final'
    data.loc[(data.T1_region == 'Y') & (data.T2_region.isin(['W', 'X'])), 'stage'] = 'final'
    data.loc[(data.T1_region == 'Z') & (data.T2_region.isin(['W', 'X'])), 'stage'] = 'final'
    data.loc[(data.T1_region == data.T2_region) & (data.T1_Seed + data.T2_Seed == 17), 'stage'] = 'Round1'
    
    fil = data.stage.isna()
    
    data.loc[fil & (data.T1_Seed.isin([1, 16])) & (data.T2_Seed.isin([8, 9])), 'stage'] = 'Round2'
    data.loc[fil & (data.T1_Seed.isin([8, 9])) & (data.T2_Seed.isin([1, 16])), 'stage'] = 'Round2'
    data.loc[fil & (data.T1_Seed.isin([5, 12])) & (data.T2_Seed.isin([4, 13])), 'stage'] = 'Round2'
    data.loc[fil & (data.T1_Seed.isin([4, 13])) & (data.T2_Seed.isin([5, 12])), 'stage'] = 'Round2'
    data.loc[fil & (data.T1_Seed.isin([6, 11])) & (data.T2_Seed.isin([3, 14])), 'stage'] = 'Round2'
    data.loc[fil & (data.T1_Seed.isin([3, 14])) & (data.T2_Seed.isin([6, 11])), 'stage'] = 'Round2'
    data.loc[fil & (data.T1_Seed.isin([7, 10])) & (data.T2_Seed.isin([2, 15])), 'stage'] = 'Round2'
    data.loc[fil & (data.T1_Seed.isin([2, 15])) & (data.T2_Seed.isin([7, 10])), 'stage'] = 'Round2'
    
    fil = data.stage.isna()
    
    data.loc[fil & (data.T1_Seed.isin([1, 16, 8, 9])) & (data.T2_Seed.isin([4, 5, 12, 13])), 'stage'] = 'Round3'
    data.loc[fil & (data.T1_Seed.isin([4, 5, 12, 13])) & (data.T2_Seed.isin([1, 16, 8, 9])), 'stage'] = 'Round3'
    data.loc[fil & (data.T1_Seed.isin([3, 6, 11, 14])) & (data.T2_Seed.isin([2, 7, 10, 15])), 'stage'] = 'Round3'
    data.loc[fil & (data.T1_Seed.isin([2, 7, 10, 15])) & (data.T2_Seed.isin([3, 6, 11, 14])), 'stage'] = 'Round3'
    
    fil = data.stage.isna()
    
    data.loc[fil & (data.T1_Seed.isin([1, 16, 8, 9, 4, 5, 12, 13])) & 
             (data.T2_Seed.isin([3, 6, 11, 14, 2, 7, 10, 15])), 'stage'] = 'Round4'
    data.loc[fil & (data.T1_Seed.isin([3, 6, 11, 14, 2, 7, 10, 15])) & 
             (data.T2_Seed.isin([1, 16, 8, 9, 4, 5, 12, 13])), 'stage'] = 'Round4'
    
    data.loc[data.stage.isna(), 'stage'] = 'impossible'
    
    return data

In [13]:
test = add_stage(target_data)

test.head()

,Season,DayNum,Team1,Team2,target,target_points,T1_Loc,T2_Loc,ID,T1_Seed,T1_region,T2_Seed,T2_region,real_stage,Seed_diff,stage
1,2003,136,1112,1436,1,29,0,0,2003_1112_1436,1,Z,16,Z,64,-15,Round1
2,2003,136,1113,1272,1,13,0,0,2003_1113_1272,10,Z,7,Z,64,3,Round1
3,2003,136,1141,1166,1,6,0,0,2003_1141_1166,11,Z,6,Z,64,5,Round1
4,2003,136,1143,1301,1,2,0,0,2003_1143_1301,8,W,9,W,64,-1,Round1
5,2003,136,1140,1163,0,-5,0,0,2003_1140_1163,12,X,5,X,64,7,Round1


In [14]:
pd.crosstab(test.stage, test.real_stage)

real_stage,16,32,4,64,68,8,Final
stage,,,,,,,
Round1,0,0,0,544,544,0,0
Round2,0,544,0,0,0,0,0
Round3,272,0,0,0,0,0,0
Round4,0,0,0,0,0,136,0
final,0,0,0,0,0,0,34
finalfour,0,0,68,0,0,0,0
